In [1]:
import pandas as pd
import math
import re

In [2]:
re_han = re.compile("([\u4E00-\u9FD5]+)", re.U)

def txtcnt(txt):
    cnt = 0
    blocks = re_han.split(txt)
    for blk in blocks:
        if not blk:
            continue
        if re_han.match(blk):
            cnt += len(blk)
    return cnt

def tokens(txt,word_len):
    words = []
    blocks = re_han.split(txt)
    for blk in blocks:
        if not blk:
            continue
        if re_han.match(blk):
            words += ngram(blk,word_len)
    return words

def ngram(txt,word_len):
    words = []
    tmp_txt = txt
    remain_len = len(tmp_txt) - word_len
    if remain_len <= 0:
        words.append(tmp_txt)
    else:
        inx = 0
        while inx <= remain_len:
            word = tmp_txt[inx:inx+word_len]
            words.append(word)
            inx += 1
    return words

In [160]:
word_len = 8
line_inx = 1
file_name = 'comment_word_'+ str(word_len) +'.txt'
with open('comment_all.txt','r') as f \
    ,open(file_name,'w') as w:
    for line in f.readlines():
        words = tokens(line,word_len)
        for word in words:
            w.write(word + ',' + str(line_inx) +',1\n')
        line_inx += 1

In [161]:
df = pd.read_csv(file_name,header=None)
df[:2]

,0,1,2
0,所以你的意思就是,1,1
1,你并没有开过特斯,1,1


In [162]:
idf = df.filter(items=[0,1])

In [163]:
len(idf)

35716

In [164]:
idf_df = idf.drop_duplicates()

In [165]:
idf_df1 = idf_df.groupby(by=0).count()

In [166]:
idf_df1[:5]

,1
0,
一,1
一一件有价值的事,1
一万个团队也是特,1
一万步刹车失灵导,1
一万步就像她说的,1


In [167]:
tf = df.filter(items=[0,2])

In [168]:
tf_df = tf.groupby(by=0).sum()

In [169]:
tf_df[:5]

,2
0,
一,1
一一件有价值的事,1
一万个团队也是特,1
一万步刹车失灵导,1
一万步就像她说的,1


In [170]:
tf_df.sort_values(by=2,ascending=False).to_csv('词频.csv')

In [171]:
tf_idf = idf_df1.join(tf_df)

In [172]:
idf_max = tf_idf[1].max()
tf_max = tf_idf[2].max()

tf_idf['idf'] = tf_idf[1].apply(lambda x: math.log(idf_max/x))
tf_idf['tf'] = tf_idf[2].apply(lambda x: x/tf_max)

In [173]:
tf_idf['score'] = tf_idf['idf'] * tf_idf['tf']

In [174]:
tf_idf[:5]

,1,2,idf,tf,score
0,,,,,
一,1,1,2.833213,0.058824,0.16666
一一件有价值的事,1,1,2.833213,0.058824,0.16666
一万个团队也是特,1,1,2.833213,0.058824,0.16666
一万步刹车失灵导,1,1,2.833213,0.058824,0.16666
一万步就像她说的,1,1,2.833213,0.058824,0.16666


In [175]:
tf_idf1 = tf_idf.reset_index()

In [176]:
tf_idf1.sort_values(by='score',ascending=False)

,0,1,2,idf,tf,score
9641,哈哈哈哈哈哈哈哈,4,15,1.446919,0.882353,1.276693
3768,什么时候检测啊,2,6,2.140066,0.352941,0.755317
24446,码的速度开,3,5,1.734601,0.294118,0.510177
10626,多,5,6,1.223775,0.352941,0.431921
31317,问候,5,6,1.223775,0.352941,0.431921
...,...,...,...,...,...,...
25245,绝不妥协,16,16,0.060625,0.941176,0.057058
7296,出特斯拉是前驱车,16,16,0.060625,0.941176,0.057058
22906,电流过大导致特斯,16,16,0.060625,0.941176,0.057058
7978,加油,17,17,0.000000,1.000000,0.000000


In [177]:
tf_idf1[(tf_idf1[2]>=3) & (tf_idf1[1]>=3)].sort_values(by='score',ascending=False)

,0,1,2,idf,tf,score
9641,哈哈哈哈哈哈哈哈,4,15,1.446919,0.882353,1.276693
24446,码的速度开,3,5,1.734601,0.294118,0.510177
31317,问候,5,6,1.223775,0.352941,0.431921
10626,多,5,6,1.223775,0.352941,0.431921
11506,好的,3,4,1.734601,0.235294,0.408141
...,...,...,...,...,...,...
22906,电流过大导致特斯,16,16,0.060625,0.941176,0.057058
31143,鉴定出特斯拉是前,16,16,0.060625,0.941176,0.057058
25402,网电流过大导致特,16,16,0.060625,0.941176,0.057058
17023,支持你,17,17,0.000000,1.000000,0.000000


In [178]:
tf_idf1.sort_values(by='score',ascending=False).to_csv('score.csv',index=False)

In [141]:
tf_idf1[:1]

,0,1,2,idf,tf,score
0,一,1,1,2.833213,0.043478,0.123183


In [23]:
import math

math.log(10.0)

2.302585092994046